# Homework 2: Learning to Rank <a class="anchor" id="toptop"></a>

**Submission instructions**:
- The cells with the `# YOUR CODE HERE` denote that these sections are graded and you need to add your implementation.
- Please use Python 3.6.5 and `pip install -r requirements.txt` to avoid version issues.
- The notebook you submit has to have the student ids, separated by underscores (E.g., `12341234_12341234_12341234_hw1.ipynb`).
- This will be parsed by a regexp, **so please double check your filename**.
- Only one member of each group has to submit the file (**please do not compress the .ipynb file when you will submit it**) to canvas.
- **Make sure to check that your notebook runs before submission**. A quick way to do this is to restart the kernel and run all the cells.  
- Do not change the number of arugments in the given functions.
- **Please do not delete/add new cells**. Removing cells **will** lead to grade deduction. 
- Note, that you are not allowed to use Google Colab.

**Learning Goals**:
- Offline LTR
  - Learn how to implement pointwise, pairwise and listwise algorithms for learning to rank 

---
**Recommended Reading**:
  - Chris Burges, Tal Shaked, Erin Renshaw, Ari Lazier, Matt Deeds, Nicole Hamilton, and Greg Hullender. Learning to rank using gradient descent. InProceedings of the 22nd international conference on Machine learning, pages 89–96, 2005.
  - Christopher J Burges, Robert Ragno, and Quoc V Le. Learning to rank with nonsmooth cost functions. In Advances inneural information processing systems, pages 193–200, 2007
  - (Sections 1, 2 and 4) Christopher JC Burges. From ranknet to lambdarank to lambdamart: An overview. Learning, 11(23-581):81, 2010
  

Additional Resources: 
- This assignment requires knowledge of [PyTorch](https://pytorch.org/). If you are unfamiliar with PyTorch, you can go over [these series of tutorials](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)

In the previous assignment, you experimented with retrieval with different ranking functions and in addition, different document representations. 

This assignment deals directly with learning to rank (LTR). In offline LTR, You will learn how to implement methods from the three approaches associated with learning to rank: pointwise, pairwise and listwise. 


**Note:**
  - The dataset used in this assignment is +100Mb in size. You may need around 2Gb of RAM for running the whole notebook.


# Table of Contents  <a class="anchor" id="top"></a>

[Back to top](#toptop)


Table of contents:


 - [Chapter 1: Offline LTR](#o_LTR) (270 points)
     - [Section 1: Dataset and Utility](#dataU) (- points)
     - [Section 2: Pointwtise LTR](#pointwiseLTR) (55 points)
     - [Section 3: Pairwise LTR](#pairwiseLTR) (35 points)
     - [Section 4: Pairwise Speed-up RankNet](#SpairwiseLTR) (65 points)
     - [Section 5: Listwise LTR](#listwiseLTR) (60 points)
     - [Section 6: Evaluation](#evaluation1) (55 points)

In [1]:
import os
import json
import itertools
from argparse import Namespace
from collections import OrderedDict
from functools import partial


import torch
import numpy as np
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange
from torch.utils.data import Dataset, DataLoader


import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

import pandas as pd

import dataset
import evaluate


# Chapter 1: Offline LTR <a class="anchor" id="o_LTR"></a>

A typical setup of learning to rank involves a feature vector constructed using a query-document pair, and a set of relevance judgements. You are given a set of triples (`query`, `document`, `relevance grade`); where relevance grade is an *ordinal* variable  with  5  grades,  for example: {`perfect`,`excellent`,`good`,`fair`,`bad`),  typically  labeled  by human annotators.  

In this assignment, you are already given the feature vector for a given document and query pair. To access these vectors, see the following code cells (note: the dataset will be automatically downloaded & the first time the next cell runs, it will take a while!)

## Section 1: Data and Utility <a class="anchor" id="dataU"></a>

[Back to TOC](#top)

First let's get familiar with the dataset and some utility methods for our implementations.

### Section 1.1 Dataset stats

| Split Name | \# queries | \# docs | \# features |
| :- | :--: | :--: | :--: |
| train | 2735 | 85227 | 501 |
| validation | 403 | 12794 | 501 |
| test | 949 | 29881 | 501 |


In [2]:
dataset.download_dataset()
data = dataset.get_dataset()
# there is only 1 fold for this dataset 
data = data.get_data_folds()[0]
# read in the data
data.read_data()

In [3]:
print(f"Number of features: {data.num_features}")
# print some statistics
for split in ["train", "validation", "test"]:
    print(f"Split: {split}")
    split = getattr(data, split)
    print(f"\tNumber of queries {split.num_queries()}")
    print(f"\tNumber of docs {split.num_docs()}")

Number of features: 501
Split: train
	Number of queries 2735
	Number of docs 85227
Split: validation
	Number of queries 403
	Number of docs 12794
Split: test
	Number of queries 949
	Number of docs 29881


### Section 1.2 Utility classes/methods

The following cells contain code that will be useful for the assigment. 

In [4]:
# these is a useful class to create torch DataLoaders, and can be used during training
class LTRData(Dataset):
    def __init__(self, data, split):
        split = {
            "train": data.train,
            "validation": data.validation,
            "test": data.test
        }.get(split)
        assert split is not None, "Invalid split!"
        features, labels = split.feature_matrix, split.label_vector
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return self.features.size(0)

    def __getitem__(self, i):
        return self.features[i], self.labels[i]

In [5]:
## example 
train_dl = DataLoader(LTRData(data, "train"), batch_size=32, shuffle=True)
# this is how you would use it to quickly iterate over the train/val/test sets 
# - (of course, without the break statement!)
for (x, y) in train_dl:
    print(x.size(), y.size())
    break

torch.Size([32, 501]) torch.Size([32])


`evaluate_model` evaluates a model, on a given split.

In [6]:
# this function evaluates a model, on a given split
def evaluate_model(pred_fn, split, batch_size=256, print_results=False, q_level=False):
    dl = DataLoader(LTRData(data, split), batch_size=batch_size)
    all_scores = []
    all_labels = []
    for (x, y) in tqdm(dl, desc=f'Eval ({split})', leave=False):
        all_labels.append(y.squeeze().numpy())
        
        with torch.no_grad():
            output = pred_fn(x)
            all_scores.append(output.squeeze().numpy())
            
    split = {
            "train": data.train,
            "validation": data.validation,
            "test": data.test
    }.get(split)   
    results = evaluate.evaluate2(np.asarray(all_scores), np.asarray(all_labels), print_results=print_results, q_level=q_level)

    return results


In [7]:
## example 
# function that scores a given feature vector e.g a network
net = nn.Linear(501, 1)
# the evaluate method accepts a function. more specifically, a callable (such as pytorch modules) 
def notwork(x):
    return net(x)
# evaluate the function
_ = evaluate_model(notwork, "validation", print_results=True)

Eval (validation):   0%|          | 0/50 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 86.7546 (19.11080)
dcg@03: 5.5485 (3.60267)
dcg@05: 7.5332 (4.43358)
dcg@10: 11.8124 (4.61507)
dcg@20: 18.4214 (5.74107)
ndcg: 0.7003 (0.04900)
ndcg@03: 0.1956 (0.12246)
ndcg@05: 0.2042 (0.11590)
ndcg@10: 0.2354 (0.09078)
ndcg@20: 0.2818 (0.09139)
precision@01: 0.1000 (0.30000)
precision@03: 0.1267 (0.19877)
precision@05: 0.1160 (0.16536)
precision@10: 0.1320 (0.11391)
precision@20: 0.1330 (0.08753)
recall@01: 0.0048 (0.01558)
recall@03: 0.0158 (0.02620)
recall@05: 0.0239 (0.03337)
recall@10: 0.0577 (0.05146)
recall@20: 0.1250 (0.09413)
relevant rank: 108.1685 (72.95383)
relevant rank per query: 2554.9400 (1450.76495)


C:\Users\lcu1812\Anaconda3\envs\IR\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


The next cell is used to generate reproducible results:

In [8]:
# use to get reproducible results
def seed(random_seed):
    import random
    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    

## Section 2: Pointwise LTR (55 points) <a class="anchor" id="pointwiseLTR"></a>

[Back to TOC](#top)

Let $x \in \mathbb{R}^d$ be an input feature vector, containing features for a query-document pair. Let $f: \mathbb{R}^d \rightarrow \mathbb{R} $ be a function that maps this feature vector to a number $f(x)$ - either a relevance score (regression) or label (classification). The data $\{x \}$ are treated as feature vectors and the relevance judgements are treated as the target which we want to predict. 

In this section, you will implement a simple Pointwise model using either a regression loss, and use the train set to train this model to predict the relevance score. 


### Section 2.1: Neural Model (25 points)

In the following cell, you will implement a simple pointwise LTR model: 
- Use a neural network to learn a model with different loss functions, using the relevance grades as the label. Use the following parameters: 
  - Layers: $501 (input) \rightarrow 256 \rightarrow 1$ where each layer is a linear layer (`nn.Linear`) with a ReLu activation function (`nn.ReLU`) in between the layers. Use the default weight initialization scheme. (Hint: use `nn.Sequential` for a one-line forward function!)
  - This network will also be used by other methods i.e Pairwise 
  
You should implement the following three methods:
- `__init__` (4 points)
- `forward` (1 point)



In [9]:
# TODO: Implement this! (5 points)
class NeuralModule(nn.Module):
    def __init__(self):
        """
        Initializes the Pointwise neural network.
        """
        
        # YOUR CODE HERE
        super(NeuralModule, self).__init__()
        self.input_size = 501
        self.batch_size = 256

        # self.model = nn.Sequential(
        #   nn.Linear(self.input_size, self.batch_size),
        #   nn.ReLU(),
        #   nn.Linear(self.batch_size, 1),
        #   nn.Softmax()
        # )

        self.model = nn.Sequential(
          nn.Linear(self.input_size, self.batch_size),
          nn.ReLU(),
          nn.Linear(self.batch_size, 1)
        )

        # self.model = nn.Sequential(
        #   nn.Linear(self.input_size, self.batch_size),
        #   nn.ReLU(),
        #   nn.Linear(self.batch_size, 1),
        #   nn.ReLU()
        # )
    
    def forward(self, x):
        """
        Takes in an input feature matrix of size (N, 501) and produces the output 
        Input: x: a [N, 501] tensor
        Output: a [N, 1] tensor
        """
        # YOUR CODE HERE
        return self.model(x)
        

In [10]:
# check the network configuration - layer dimensions and configurations
point_nn_reg = NeuralModule()
print(point_nn_reg)

NeuralModule(
  (model): Sequential(
    (0): Linear(in_features=501, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1, bias=True)
  )
)


In [11]:
# test the forward function
n = 10
inp = torch.rand(n, data.num_features)
out = point_nn_reg(inp)


In [12]:
print(inp.grad)
print(out)

None
tensor([[0.1734],
        [0.1162],
        [0.1592],
        [0.1625],
        [0.1623],
        [0.0750],
        [0.0011],
        [0.0645],
        [0.0691],
        [0.0936]], grad_fn=<AddmmBackward>)


**Implementation (20 points):**
Implement `train_batch` function to compute the gradients (`backward()` function) and update the weights (`step()` function), using the specified loss function.

In [13]:
# TODO: Implement this! (20 points)

def train_batch(net, x, y, loss_fn, optimizer):
    """
    Takes as input a batch of size N, i.e. feature matrix of size (N, 501), label vector of size (N), the loss function and optimizer for computing the gradients, 
    and updates the weights of the model.

    Input:  x: feature matrix, a [N, 501] tensor
            y: label vector, a [N] tensor
            loss_fn: an implementation of a loss function
            optimizer: an optimizer for computing the gradients (we use Adam)
    """
    # YOUR CODE HERE
    optimizer.zero_grad()
    prediction = net(x)
    loss = loss_fn(prediction, y)  # add requires_grad=True?
    net.zero_grad()  # not sure if this is necessary => Lan: i think maybe not
    loss.backward()
    optimizer.step()


In [14]:
# Please do not change this. This cell is used for grading.


\#### Please do not change this. This cell is used for grading.

### Section 2.2: Loss Functions (5 points)
Pointwise LTR algorithms use pointwise loss functions.
Usually, the popular loss functions for pointwise LTR is Regression loss.

**Implementation (5 points):**
Implement regression loss.

In [15]:
# TODO: Implement this! (5 points)
def pointwise_loss(output, target):
    """
    Regression loss - returns a single number. 
    Make sure to use the MSE loss
    output: (float) tensor, shape - [N, 1] 
    target: (float) tensor, shape - [N]. 
    """
    assert target.dim() == 1
    assert output.size(0) == target.size(0)
    assert output.size(1) == 1
    
    # YOUR CODE HERE
    target = target.view(1, -1)
    criterion = nn.MSELoss()
    loss = criterion(output, target)
    # print(loss.grad_fn)  # MSELoss
    # print(loss.grad_fn.next_functions[0][0])  # Linear
    # print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU
    return loss


In [16]:
## Test pointwise_loss
g = torch.manual_seed(42)
output = [torch.randint(low=0, high=5, size=(5, 1), generator=g).float() for _ in range(5)]
target = torch.randint(low=0, high=5, size=(5,), generator=g).float()

l = [pointwise_loss(o, target).item() for o in output]
print(f'your results:{l}')


your results:[3.200000047683716, 6.360000133514404, 4.159999847412109, 5.400000095367432, 3.240000009536743]


C:\Users\lcu1812\Anaconda3\envs\IR\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1, 5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


**Implementation (25 points):**
Now implement a wrapper for training a pointwise LTR, that takes the model as input and trains the model.

**Rubric:**
 - Network is trained for specified epochs, and iterates over the entire dataset and (train) data is shuffled : 5 points
 - Evaluation on the validation set: 5 points
 - Performance as expected: 15 points

In [17]:
# TODO: Implement this! (25 points)
def train_pointwise(net, params):
    """
    This function should train a Pointwise network. 
    
    The network is trained using the Adam optimizer
        
    
    Note: Do not change the function definition! 
    
    
    Hints:
    1. Use the LTRData class defined above
    2. Do not forget to use net.train() and net.eval()
    
    Inputs:
            net: the neural network to be trained

            params: params is an object which contains config used in training 
                (eg. params.epochs - the number of epochs to train). 
                For a full list of these params, see the next cell. 
    
    Returns: a dictionary containing: "metrics_val" (a list of dictionaries) and 
             "metrics_train" (a list of dictionaries). 
             
             "metrics_val" should contain metrics (the metrics in params.metrics) computed
             after each epoch on the validation set (metrics_train is similar). 
             You can use this to debug your models.
    
    """
    
    
    val_metrics_epoch = []
    train_metrics_epoch = []
    optimizer = Adam(net.parameters(), lr=params.lr)
    loss_fn = pointwise_loss
    
    # YOUR CODE HERE

    for i in range(params.epochs):
      train_dl = DataLoader(LTRData(data, "train"), batch_size=params.batch_size, shuffle=True)
      for (x, y) in train_dl:
        net.input_size = x.size()[1]
        net.batch_size = params.batch_size
        break

      for (x, y) in train_dl:
          train_batch(net, x, y, loss_fn, optimizer)

      train_metrics_epoch.append(evaluate_model(net, "train", print_results=True))

      # validation
      val_dl = DataLoader(LTRData(data, "validation"), batch_size=params.batch_size, shuffle=True)

      val_metrics_epoch.append(evaluate_model(net, "validation", print_results=True))
    
    return {
        "metrics_val": val_metrics_epoch,
        "metrics_train": train_metrics_epoch
    }

In [18]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [19]:
# Change this to test your code!
pointwise_test_params = Namespace(epochs=1, 
                    lr=1e-3,
                    batch_size=256,
                   metrics={"ndcg"})
# uncomment to test your code
# train a regression model
met_reg = train_pointwise(point_nn_reg, pointwise_test_params)


C:\Users\lcu1812\Anaconda3\envs\IR\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1, 256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\lcu1812\Anaconda3\envs\IR\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1, 235])) that is different to the input size (torch.Size([235, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Eval (train):   0%|          | 0/333 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 84.1757 (20.46565)
dcg@03: 4.2110 (3.96153)
dcg@05: 5.9614 (4.61628)
dcg@10: 9.1291 (5.27677)
dcg@20: 14.6216 (6.94544)
ndcg: 0.6790 (0.05633)
ndcg@03: 0.1556 (0.14253)
ndcg@05: 0.1669 (0.12585)
ndcg@10: 0.1860 (0.10459)
ndcg@20: 0.2237 (0.09698)
precision@01: 0.0931 (0.29056)
precision@03: 0.0941 (0.18218)
precision@05: 0.1027 (0.15468)
precision@10: 0.0934 (0.10376)
precision@20: 0.1047 (0.09170)
recall@01: 0.0041 (0.01510)
recall@03: 0.0126 (0.02902)
recall@05: 0.0236 (0.03910)
recall@10: 0.0410 (0.04777)
recall@20: 0.0925 (0.07526)
relevant rank: 118.3823 (71.87130)
relevant rank per query: 2812.3784 (1480.68271)


Eval (validation):   0%|          | 0/50 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 83.0985 (20.50176)
dcg@03: 3.8647 (3.46365)
dcg@05: 5.5414 (4.18622)
dcg@10: 8.4263 (5.02305)
dcg@20: 13.2660 (5.89523)
ndcg: 0.6681 (0.06118)
ndcg@03: 0.1383 (0.12349)
ndcg@05: 0.1517 (0.11437)
ndcg@10: 0.1697 (0.10368)
ndcg@20: 0.2014 (0.08982)
precision@01: 0.0400 (0.19596)
precision@03: 0.0667 (0.17638)
precision@05: 0.0720 (0.14288)
precision@10: 0.0760 (0.10874)
precision@20: 0.0870 (0.08174)
recall@01: 0.0013 (0.00621)
recall@03: 0.0080 (0.02099)
recall@05: 0.0150 (0.03208)
recall@10: 0.0305 (0.04360)
recall@20: 0.0729 (0.06082)
relevant rank: 117.0008 (70.94381)
relevant rank per query: 2763.5600 (1360.98517)


The next cell is used to generate results:

In [20]:

    
def create_results(net, train_fn, prediction_fn, *train_params):
    
    print("Training Model")
    metrics = train_fn(net, *train_params)
    net.eval()
    test_metrics, test_qq = evaluate_model(prediction_fn, "test", print_results=True, q_level=True)
    
    
    test_q = {}
    for m in {"ndcg", "precision@05", "recall@05"}:
        test_q[m] = test_qq[m]
    
    
    return {
        "metrics": metrics,
        "test_metrics": test_metrics,
        "test_query_level_metrics": test_q,
        }


Now use the above functions to generate your results:

In [21]:
seed(42)
params_regr = Namespace(epochs=11, 
                    lr=1e-3,
                    batch_size=256,
                    metrics={"ndcg", "precision@05", "recall@05"})

pointwise_regression_model = NeuralModule()
pw_regr = create_results(pointwise_regression_model, 
                          train_pointwise, 
                          pointwise_regression_model,
                          params_regr)

Training Model


Eval (train):   0%|          | 0/333 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 87.6884 (20.76704)
dcg@03: 5.9154 (4.68019)
dcg@05: 8.1565 (5.20717)
dcg@10: 12.2291 (6.24992)
dcg@20: 18.7541 (7.66434)
ndcg: 0.7081 (0.05587)
ndcg@03: 0.2199 (0.17480)
ndcg@05: 0.2290 (0.14949)
ndcg@10: 0.2492 (0.12651)
ndcg@20: 0.2870 (0.10587)
precision@01: 0.1682 (0.37402)
precision@03: 0.1572 (0.22145)
precision@05: 0.1604 (0.17430)
precision@10: 0.1471 (0.12956)
precision@20: 0.1462 (0.10161)
recall@01: 0.0082 (0.02373)
recall@03: 0.0216 (0.03523)
recall@05: 0.0373 (0.04551)
recall@10: 0.0679 (0.06329)
recall@20: 0.1301 (0.08611)
relevant rank: 105.7271 (70.91133)
relevant rank per query: 2511.7327 (1360.05899)


Eval (validation):   0%|          | 0/50 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 85.9730 (20.84005)
dcg@03: 4.3483 (3.72987)
dcg@05: 6.1761 (4.59519)
dcg@10: 10.1527 (5.07312)
dcg@20: 16.6332 (6.36849)
ndcg: 0.6916 (0.05844)
ndcg@03: 0.1577 (0.13644)
ndcg@05: 0.1684 (0.12798)
ndcg@10: 0.2017 (0.10473)
ndcg@20: 0.2513 (0.09526)
precision@01: 0.0800 (0.27129)
precision@03: 0.0667 (0.16330)
precision@05: 0.0840 (0.14472)
precision@10: 0.1020 (0.10098)
precision@20: 0.1170 (0.08866)
recall@01: 0.0032 (0.01085)
recall@03: 0.0088 (0.02125)
recall@05: 0.0171 (0.03215)
recall@10: 0.0423 (0.04047)
recall@20: 0.1019 (0.06999)
relevant rank: 105.3480 (70.62151)
relevant rank per query: 2488.3200 (1314.38955)


...

"metric": "mean" ("standard deviation")
dcg: 87.9466 (21.23779)
dcg@03: 6.3259 (5.40883)
dcg@05: 8.5546 (5.98348)
dcg@10: 12.9180 (7.01506)
dcg@20: 19.3295 (8.39612)
ndcg: 0.7096 (0.05927)
ndcg@03: 0.2304 (0.19069)
ndcg@05: 0.2371 (0.16150)
ndcg@10: 0.2604 (0.13610)
ndcg@20: 0.2946 (0.11648)
precision@01: 0.1922 (0.39402)
precision@03: 0.1862 (0.25449)
precision@05: 0.1748 (0.20051)
precision@10: 0.1652 (0.14010)
precision@20: 0.1538 (0.10867)
recall@01: 0.0091 (0.02568)
recall@03: 0.0268 (0.04386)
recall@05: 0.0401 (0.05118)
recall@10: 0.0751 (0.06664)
recall@20: 0.1349 (0.09059)
relevant rank: 105.3908 (72.38662)
relevant rank per query: 2503.7447 (1337.17682)


Eval (validation):   0%|          | 0/50 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 87.6349 (21.25735)
dcg@03: 6.9059 (6.14486)
dcg@05: 9.1315 (6.83678)
dcg@10: 12.9486 (7.53471)
dcg@20: 19.2844 (9.21252)
ndcg: 0.7056 (0.05521)
ndcg@03: 0.2372 (0.20274)
ndcg@05: 0.2411 (0.17379)
ndcg@10: 0.2531 (0.14205)
ndcg@20: 0.2867 (0.12056)
precision@01: 0.2000 (0.40000)
precision@03: 0.1933 (0.25897)
precision@05: 0.1840 (0.21850)
precision@10: 0.1460 (0.13448)
precision@20: 0.1440 (0.11603)
recall@01: 0.0112 (0.02643)
recall@03: 0.0304 (0.04960)
recall@05: 0.0419 (0.05410)
recall@10: 0.0681 (0.07116)
recall@20: 0.1231 (0.08134)
relevant rank: 105.3641 (72.48283)
relevant rank per query: 2488.7000 (1242.87969)


Eval (test):   0%|          | 0/117 [00:00<?, ?it/s]

"metric": "mean" ("standard deviation")
dcg: 91.5237 (19.95419)
dcg@03: 5.9796 (5.13690)
dcg@05: 8.0646 (5.62469)
dcg@10: 12.2020 (6.20981)
dcg@20: 18.8128 (7.31521)
ndcg: 0.7076 (0.05221)
ndcg@03: 0.2131 (0.18543)
ndcg@05: 0.2174 (0.15247)
ndcg@10: 0.2384 (0.11516)
ndcg@20: 0.2790 (0.09887)
precision@01: 0.1709 (0.37646)
precision@03: 0.1510 (0.22852)
precision@05: 0.1368 (0.16672)
precision@10: 0.1342 (0.11259)
precision@20: 0.1423 (0.09190)
recall@01: 0.0077 (0.01869)
recall@03: 0.0205 (0.03398)
recall@05: 0.0307 (0.04566)
recall@10: 0.0562 (0.05457)
recall@20: 0.1139 (0.07329)
relevant rank: 109.4135 (71.59997)
relevant rank per query: 2815.7607 (1384.14820)


In [22]:
assert "test_metrics" in pw_regr.keys()
assert "ndcg" in pw_regr["test_metrics"].keys()
assert "precision@05" in pw_regr["test_metrics"].keys()
assert "recall@05" in pw_regr["test_metrics"].keys()

## Section 3: Pairwise LTR (35 points) <a class="anchor" id="pairwiseLTR"></a>

[Back to TOC](#top)

In this section,  you will learn and implement RankNet, a  pairwise learning to rank algorithm.

For a given query, consider two documents $D_i$ and $D_j$ with two different ground truth relevance  labels,  with  feature  vectors $x_i$ and $x_j$ respectively.   The  RankNet  model,  just  like  the pointwise model, uses $f$ to predict scores i.e $s_i=f(x_i)$ and $s_j=f(x_j)$, but uses a different loss during  training. $D_i \triangleright D_j$ denotes  the  event  that $D_i$ should  be  ranked  higher  than $D_j$.   The  two outputs $s_i$ and $s_j$ are mapped to a learned probability that $D_i \triangleright D_j$: 


$$        P_{ij} = \frac{1}{1 + e^{-\sigma(s_i - s_j)}} $$
  
where $\sigma$ is a parameter that determines the shape of the sigmoid. The loss of the RankNet model is the cross entropy cost function:

$$        C = - \bar{P}_{ij} \log P_{ij} - (1-\bar{P}_{ij}) \log (1 - P_{ij}) $$

As the name suggests, in the pairwise approach to LTR, we optimize a loss $l$ over pairs of documents. Let $S_{ij} \in \{0, \pm1 \}$ be equal to $1$ if the relevance of document $i$ is greater than document $j$; $-1$ if document $j$ is more relevant than document $i$; and 0 if they have the same relevance. This gives us $\bar{P}_{ij} = \frac{1}{2} (1 + S_{ij})$ so that $\bar{P}_{ij} = 1$ if $D_i \triangleright D_j$; $\bar{P}_{ij} = 0$ if $D_j \triangleright D_i$; and finally $\bar{P}_{ij} = \frac{1}{2}$ if the relevance is identical. This gives us:

$$        C = \frac{1}{2}(1- S_{ij})\sigma(s_i - s_j) + \log(1+ e^{-\sigma(s_i - s_j)}) $$

Now, consider a single query for which $n$ documents have been returned. Let the output scores of the ranker be $s_j$ ; $j=\{1, \dots, n \}$, the model parameters be $w_k \in \mathbb{R}^W$, and let the set of pairs of document indices used for training be $\mathcal{P}$. Then, the total cost is $C_T = \sum_{i,j \in \mathcal{P}} C(s_i; s_j)$. 



- Implement RankNet. You should construct training samples by creating all possible pairs of documents for a given query and optimizing the loss above. Use the following parameters:
  - Layers: $501 (input) \rightarrow 256 \rightarrow 1$, where each layer is a linear layer (`nn.Linear`) with a ReLu activation function (`nn.ReLu`) in between the layers. Use the default weight initialization scheme. (Hint: use `nn.Sequential` for a one-line forward function!)

For the pairwise loss, we need to have a structured **dataloader** which detects the documents associated with a specific query:

In [23]:
class QueryGroupedLTRData(Dataset):
    def __init__(self, data, split):
        self.split = {
            "train": data.train,
            "validation": data.validation,
            "test": data.test
        }.get(split)
        assert self.split is not None, "Invalid split!"
    
    def __len__(self):
        return self.split.num_queries()

    def __getitem__(self, q_i):
        feature = torch.FloatTensor(self.split.query_feat(q_i))
        labels = torch.FloatTensor(self.split.query_labels(q_i))
        return feature, labels

    
## example
train_data = QueryGroupedLTRData(data, "train")
# this is how you would use it to quickly iterate over the train/val/test sets 

q_i = 300
features_i, labels_i = train_data[q_i]
print(f"Query {q_i} has {len(features_i)} query-document pairs")
print(f"Shape of features for Query {q_i}: {features_i.size()}")
print(f"Shape of labels for Query {q_i}: {labels_i.size()}")
        

Query 300 has 57 query-document pairs
Shape of features for Query 300: torch.Size([57, 501])
Shape of labels for Query 300: torch.Size([57])


**Implementation (35 points):**
First, implement the pairwaise loss, described above.

**Rubric:**
 - Each ordering <i,j> combination is considered: 10 points
 - Proper application of the formula: 10 points
 - Mean loss: 5 points
 - Loss values for test cases as expected: 10 points

In [24]:
# TODO: Implement this! (35 points)
def pairwise_loss(scores, labels):
    """
    Compute and return the pairwise loss *for a single query*. To compute this, compute the loss for each 
    ordering in a query, and then return the mean. Use sigma=1.
    
    For a query, consider all possible ways of comparing 2 document-query pairs.
    
    Hint: See the next cell for an example which should make it clear how the inputs look like
    
    scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
    labels: tensor of size [N], contains the relevance labels 
    """
        
    
    
    # if there's only one rating
    if labels.size(0) < 2:
        return None
    
    count = 0
    # C_list = []
    C_matrix = torch.zeros([len(scores), len(scores)])
    S_ij = {}
    for i_index, i_label  in enumerate(labels):
        for j_index, j_label in enumerate(labels):
            if i_label>j_label:
                S_ij[(i_index,j_index)] = 1
            if i_label<j_label:
                S_ij[(i_index,j_index)] = -1
            if i_label==j_label:
                S_ij[(i_index,j_index)]= 0
    for x in range(len(scores)):
        for y in range(len(scores)):
            if x != y:
                C = ((0.5)*(1-S_ij[x,y])*(scores[x]-scores[y]))+np.log(1+np.exp(-1*(scores[x]-scores[y])))
                # C_list.append(C)
                C_matrix[x, y] = C
                count+=1

                
    
    # mean = sum(C_list)/len(C_list)
    mean_matrix = C_matrix.sum()/count
    
    return mean_matrix
    

In [25]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [26]:
scores_1 = torch.FloatTensor([0.2, 2.3, 4.5, 0.2, 1.0]).unsqueeze(1)
labels_1 = torch.FloatTensor([1, 2, 3, 0, 4])


scores_2 = torch.FloatTensor([3.2, 1.7]).unsqueeze(1)
labels_2 = torch.FloatTensor([3, 1])

assert torch.allclose(pairwise_loss(scores_1, labels_1), torch.tensor(0.6869), atol=1e-03)
assert torch.allclose(pairwise_loss(scores_2, labels_2), torch.tensor(0.2014), atol=1e-03)


## Section 4: Pairwise: Speed-up RankNet (65 points) <a class="anchor" id="SpairwiseLTR"></a>

[Back to TOC](#top)

To speed up training of the previous model, we can consider a sped up version of the model, where instead of `.backward` on the loss, we use `torch.backward(lambda_i)`. 

The derivative of the total cost $C_T$ with respect to the model parameters $w_k$ is:

$$        \frac{\partial C_T}{\partial w_k} = \sum_{(i,j) \in \mathcal{P}} \frac{\partial C(s_i, s_j)}{\partial s_i} \frac{\partial s_i}{\partial w_k} + \frac{\partial C(s_i, s_j)}{\partial s_j} \frac{\partial s_j}{\partial w_k} $$

We can rewrite this sum by considering the set of indices $j$ , for which $\{i,j\}$ is a valid pair, denoted by $\mathcal{P}_i$, and the set of document indices $\mathcal{D}$:

$$
\frac{\partial C_T}{\partial w_k} = \sum_{i \in \mathcal{D}}
\frac{\partial s_i}{\partial w_k} \sum_{j \in \mathcal{P}_i} 
\frac{\partial C(s_i, s_j)}{\partial s_i} 
$$

This sped of version of the algorithm first computes scores $s_i$ for all the documents. Then for each $j= 1, \dots, n$, compute:

$$
\lambda_{ij} = \frac{\partial C(s_i, s_j)}{\partial s_i} = \sigma \bigg( \frac{1}{2}(1 - S_{ij}) -  \frac{1}{1 + e^{\sigma(s_i -s_j))}} \bigg) \\
\lambda_i = \sum_{j \in \mathcal{P}_i} \frac{\partial C(s_i, s_j)}{\partial s_i} = \sum_{j \in \mathcal{P}_i} \lambda_{ij}
$$

That gives us:

$$
\frac{\partial C_T}{\partial w_k} = \sum_{i \in \mathcal{D}}
\frac{\partial s_i}{\partial w_k} \lambda_i
$$

This can be directly optimized in pytorch using: `torch.autograd.backward(scores, lambda_i)` 
 


**Implementation (50 points):**
Implement the sped-up version of pairwise loss, described above.

**Rubric:**
 - Each ordering <i,j> combination is considered: 15 points
 - Proper application of the formula: 15 points
 - Loss values for test cases as expected: 20 points

In [27]:
# TODO: Implement this! (50 points)
def compute_lambda_i(scores, labels):
    """
    Compute \lambda_i (defined in the previous cell). (assume sigma=1.)
    
    scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
    labels: tensor of size [N], contains the relevance labels 
    
    return: \lambda_i, a tensor of shape: [N, 1]
    """
        
    # YOUR CODE HERE
    N = len(labels)
    labels = labels[None, :]
    sij = labels.T - labels
    sij[sij<-1] = -1
    sij[sij>1] = 1
    diff_scores = scores - scores.T

    part1 = torch.exp(diff_scores)
    part2 = part1.add(1)
    part3 = 1 / part2

    part4 = 1 - sij
    part5 = 0.5 * part4

    lambda_i_j = part5 - part3

    lambda_i = torch.sum(lambda_i_j, 1)
    lambda_i = torch.reshape(lambda_i, (N, 1))
    
    return lambda_i

In [28]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [29]:
def mean_lambda(scores, labels):
    return torch.stack([compute_lambda_i(scores, labels).mean(), torch.square(compute_lambda_i(scores, labels)).mean()])

scores_1 = torch.FloatTensor([10.2, 0.3, 4.5, 2.0, -1.0]).unsqueeze(1)
labels_1 = torch.FloatTensor([1, 2, 3, 0, 4])

assert torch.allclose(mean_lambda(scores_1, labels_1), torch.tensor([0,5.5072]), atol=1e-03)

scores_2 = torch.FloatTensor([3.2, 1.7]).unsqueeze(1)
labels_2 = torch.FloatTensor([3, 1])

assert torch.allclose(mean_lambda(scores_2, labels_2), torch.tensor([0,3.3279e-02]), atol=1e-03)



**Implementation (15 points):**
Implement `train_batch_vector` function to compute the gradients (`torch.autograd.backward(scores, lambda_i)` function) and update the weights (`step()` function), using the specified loss function.

In [30]:
# TODO: Implement this! (15 points)

def train_batch_vector(net, x, y, loss_fn, optimizer):
    """
    Takes as input a batch of size N, i.e. feature matrix of size (N, 501), label vector of size (N), the loss function and optimizer for computing the gradients, and updates the weights of the model.
    The loss function returns a vector of size [N, 1], the same as the output of network.

    Input:  x: feature matrix, a [N, 501] tensor
            y: label vector, a [N] tensor
            loss_fn: an implementation of a loss function
            optimizer: an optimizer for computing the gradients (we use Adam)
    """
    # YOUR CODE HERE
    optimizer.zero_grad()
    scores = net(x)
    loss = loss_fn(scores, y)
    print('scores: ', scores)
    print('loss: ', loss)
    torch.autograd.backward(scores, loss)
    optimizer.step()

In [31]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

##  Section 5: Listwise LTR (60 points) <a class="anchor" id="listwiseLTR"></a>

[Back to TOC](#top)

In this section, you will implement LambdaRank, a listwise approach to LTR. Consider the computation of $\lambda$ for sped-up RankNet (that you've already implemented). $\lambda$ here amounts to the 'force' on a document given its neighbours in the ranked list. The design of $\lambda$ in LambdaRank is similar to RankNet, but is scaled by DCG gain from swapping the two documents in question. Let's suppose that the corresponding ranks of doucment $D_i$ and $D_j$ are $r_i$ and $r_j$ respectively. Given a ranking measure $IRM$, such as $NDCG$ or $ERR$, the lambda function in LambdaRank is defined as:


$$        \frac{\partial C}{\partial s_i} = \sum_{j \in D} \lambda_{ij} \cdot |\bigtriangleup IRM (i,j)| $$

Where $|\bigtriangleup IRM(i,j)|$ is the absolute difference in $IRM$ after swapping the rank positions $r_i$ and $r_j$ while leaving everything else unchanged ($| \cdot |$ denotes the absolute value). Note that we do not backpropogate $|\bigtriangleup IRM|$, it is treated as a constant that scales the gradients. In this assignment we will use $|\bigtriangleup NDCG|$

**Implementation (60 points):**
Implement the listwise loss.

**Rubric:**
 - Each ordering <i,j> combination is considered: 15 points
 - Computing $|\bigtriangleup NDCG|$: 15 points 
 - Proper application of the formula: 15 points 
 - Loss values as expected: 15 points

In [32]:
# TODO: Implement this! (60 points)
# def listwise_loss(scores, labels):
    
#     """
#     Compute the LambdaRank loss. (assume sigma=1.)
    
#     scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
#     labels: tensor of size [N], contains the relevance labels 
    
#     returns: a tensor of size [N, 1]
#     """
#     print('scores: ', scores)
#     print('labels: ', labels)
    
    # S_ij = {}
    # swap_t = {}
    # for i_index, i_label  in enumerate(labels):
    #     for j_index, j_label in enumerate(labels):
    #         swap_t[i_index,j_index] = swap_doc(labels,i_index,j_index)
    #         if i_label > j_label:
    #             S_ij[(i_index,j_index)] = 1
    #         if i_label < j_label:
    #             S_ij[(i_index,j_index)] = -1
    #         if i_label == j_label:
    #             S_ij[(i_index,j_index)]= 0
    # N = len(labels)
    # DOC = []
    # for x in range(len(labels)):
    #     DOC.append((2**labels[x]-1)/np.log2(x+2))
            
    # lambda_i_j = torch.zeros([N,N])
    # listwise_loss = torch.zeros([N,N])
    # # lambda_i_j = torch.zeros([N,N]).cuda()
    # # listwise_loss = torch.zeros([N,N]).cuda()
    # Ideal_DOC = IDOC(labels)
    
    
    
    # DOC_swap_t_d = {}
    # for i in swap_t:
    #     DOC_swap_t = []
    #     for x in range(len(labels)):
    #         DOC_swap_t.append((2**swap_t[i][x]-1)/np.log2(x+2))
    #     DOC_swap_t_d[i] = sum(DOC_swap_t)
            
            
    # for i in range(len(labels)):
    #     for j in range(len(labels)):
    #         lambda_i_j[i,j] = (1/2*(1-S_ij[(i,j)])-1/(1+torch.exp(scores[i][0]-scores[j][0])))
    #         IRM = abs((sum(DOC)/Ideal_DOC)-(DOC_swap_t_d[i,j]/Ideal_DOC))
    #         listwise_loss[i,j] = lambda_i_j[i,j] * IRM
    # final = torch.sum(listwise_loss,1)
    # final = torch.reshape(final,(N,1))

    # N = len(labels)

    # for i in range(N):
    #   for j in range(N):

    #     # calculate total before swapping
    #     dcg_i_before = (2**labels[i]-1)/np.log2(i+2)
    #     dcg_j_before = (2**labels[j]-1)/np.log2(j+2)
    #     total_before = dcg_i_before + dcg_j_before

    #     # calculate total after swapping
    #     dcg_i_after = (2**labels[i]-1)/np.log2(j+2)
    #     dcg_j_after = (2**labels[j]-1)/np.log2(i+2)
    #     total_after = dcg_i_after + dcg_j_after

    #     # calculate difference
    #     delta_dcg = abs(total_before - total_after)
    #   lambda_rank_net = compute_lambda_rank_net(scores, labels)

    #   lambda_rank.append(lambda_rank_net * delta_ndcg)
    # sorted_labels = labels.sort(descending=True)[0][:]
    # delta_ndcg = get_delta_ndcg(sorted_labels, labels, gpu=False)
    # lambda_i = compute_lambda_i(scores, labels)
    # lambda_rank = torch.mul(lambda_i, delta_ndcg)

    # N = len(labels)
    # lambda_i_j = torch.zeros([N,N])
    # ideal_labels = labels.sort(descending=True)[0][:]
    # print('ideal_labels: ', ideal_labels)
    # lambda_rank = []
    
    # for i, li  in enumerate(labels):
    #     for j, lj in enumerate(labels):
    #         if li>lj:
    #             Sij = 1
    #         if li<lj:
    #             Sij = -1
    #         if li==lj:
    #             Sij = 0
    #         lambda_i_j[i,j] = 1*(1/2*(1-Sij)-1/(1+torch.exp(scores[i][0]-scores[j][0])))  # all variables are matrices

    #         print('i: ', i)
    #         print('j: ', j)
    #         # swap

    #         swapped_labels = labels.tolist()
    #         print('to list: ', swapped_labels)
    #         swapped_labels[i] = labels[j]
    #         swapped_labels[j] = labels[i]
    #         swapped_labels = torch.tensor(swapped_labels)

    #         print('labels: ', labels)
    #         print('swapped_labels: ', swapped_labels)

    #         unswapped_ndcg = evaluate.ndcg_at_k(labels, ideal_labels, N)
    #         swapped_ndcg = evaluate.ndcg_at_k(swapped_labels, ideal_labels, N)
    #         delta_ndcg = unswapped_ndcg - swapped_ndcg

    #         lambda_rank.append(lambda_i_j * abs(delta_ndcg))  # [N, N] * [N, N]

    

    # return lambda_rank

In [33]:
# TODO: Implement this! (60 points)
def listwise_loss(scores, labels):
    
    """
    Compute the LambdaRank loss. (assume sigma=1.)
    
    scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
    labels: tensor of size [N], contains the relevance labels 
    
    returns: a tensor of size [N, 1]
    """

    N = len(labels)
    # labels = labels[None, :]
    # sij = labels.T - labels
    # sij[sij<-1] = -1
    # sij[sij>1] = 1
    # diff_scores = scores - scores.T

    # part1 = torch.exp(diff_scores)
    # part2 = part1.add(1)
    # part3 = 1 / part2

    # part4 = 1 - sij
    # part5 = 0.5 * part4
    # lambda_i_j = part5 - part3

    # print('labels: ', labels)
    ideal_labels = labels.sort(descending=True)[0][:].squeeze()
    # print('ideal: ', ideal_labels)

    # ndcg = torch.zeros([N, N]) + ndcg_at_k(labels, ideal_labels, N)
    # print('ndcg: ', ndcg)
    # delta_ndcg = get_delta_ndcg(ideal_labels, labels, gpu=False)
    # swapped_ndcg = 

    # swap
    # swapped_labels = labels.tolist()
    # print('to list: ', swapped_labels)
    # swapped_labels[i] = labels[j]
    # swapped_labels[j] = labels[i]
    # swapped_labels = torch.tensor(swapped_labels)

    # print('delta_ndcg: ', delta_ndcg)

    # return delta_ndcg

    ideal_labels = labels.sort(descending=True)[0][:].squeeze()
    ideal_labels = ideal_labels[None, :]
    labels = labels[None, :]

    print('labels: ', labels)
    print('ideal: ', ideal_labels)

    get_delta_ndcg(batch_stds=ideal_labels, batch_stds_sorted_via_preds=labels)

    pass

In [34]:
def dcg_at_k(sorted_labels, k):
  denom = 1./torch.log2(torch.arange(k)+2.)
  nom = 2**sorted_labels-1.
  part1 = nom[:k]*denom
  dcg = torch.sum(part1)
  return dcg
  
def ndcg_at_k(sorted_labels, ideal_labels, k):
  return dcg_at_k(sorted_labels, k) / dcg_at_k(ideal_labels, k)

In [35]:
scores = torch.FloatTensor([10.2, 0.3, 4.5, 2.0, -1.0]).unsqueeze(1)
labels = torch.FloatTensor([1, 2, 3, 0, 4])
listwise_loss(scores, labels)

labels:  tensor([[1., 2., 3., 0., 4.]])
ideal:  tensor([[4., 3., 2., 1., 0.]])


NameError: name 'get_delta_ndcg' is not defined

In [ ]:
listwise_loss(scores, labels)

In [ ]:
a = torch.tensor([1, 2, 3, 0, 4])
a = a[None, :]
b = torch.tensor([4, 3, 2, 1, 0])
b = b[:, None]
print(a - b)

In [ ]:
a = torch.tensor([1, 2, 3])
a = a[None, :]
a_star = a.T - a
a_star[a_star<-1] = -1
a_star[a_star>1] = 1
print(a_star)

b = torch.tensor([[0, 0, 1],
 [0, 0, 0],
 [-1, 0, 0]])

c = a_star + b
print(a_star)
print(c)

# 1/2*(1-Sij)-1

# [[(1, 1), (1, 2), (1, 3)],
#  [(2, 1), (2, 2), (2, 3)],
#  [(3, 1), (3, 2), (3, 3)]]



d = torch.tensor([[0.5, 0, 0],
                  [1, 0.5, 0],
                  [1, 1, 0.5]])

tensor([[ 0, -1, -1],
        [ 1,  0, -1],
        [ 1,  1,  0]])
tensor([[ 0, -1, -1],
        [ 1,  0, -1],
        [ 1,  1,  0]])
tensor([[ 0, -1,  0],
        [ 1,  0, -1],
        [ 0,  1,  0]])


In [ ]:
a1 = torch.tensor([[1, 1, 1],
 [2, 2, 2],
 [3, 3, 3]])

a2 = torch.tensor([[1, 2, 3],
  [1, 2, 3],
  [1, 2, 3]])

a3 = a1 - a2
print(a3)

tensor([[ 0, -1, -2],
        [ 1,  0, -1],
        [ 2,  1,  0]])


In [ ]:
def IDOC(labels):
    m = labels.sort(descending=True)
    list_IDOC = []
    for x in range(len(labels)):
        list_IDOC.append((2**m[0][x]-1)/np.log2(x+2))
    IDOC = sum(list_IDOC)
    return IDOC

def swap_doc(labels,i,j):
    swap_doc = []
    for x in range(len(labels)):
        swap_doc.append(labels[x])
    swap_doc[i], swap_doc[j] = swap_doc[j], swap_doc[i]
    return swap_doc

\#### Please do not change this. This cell is used for grading.

In [36]:
def mean_lambda_list(scores, labels):
    print(torch.square(listwise_loss(scores, labels)).mean())
    return torch.stack([listwise_loss(scores, labels).mean(), torch.square(listwise_loss(scores, labels)).mean()])

scores_1 = torch.FloatTensor([10.2, 0.3, 4.5, 2.0, -1.0]).unsqueeze(1)
labels_1 = torch.FloatTensor([1, 2, 3, 0, 4])
# assert torch.allclose(mean_lambda_list(scores_1, labels_1), torch.tensor([0,0.1336]), atol=1e-03)

scores_2 = torch.FloatTensor([3.2, 1.7]).unsqueeze(1)
labels_2 = torch.FloatTensor([3, 1])
assert torch.allclose(mean_lambda_list(scores_2, labels_2), torch.tensor([0,2.8024e-03]), atol=1e-03)

labels:  tensor([[3., 1.]])
ideal:  tensor([[3., 1.]])


NameError: name 'get_delta_ndcg' is not defined

## Section 6: Comparing Pointwise, Pairwise and Listwise (55 points) <a class="anchor" id="evaluation1"></a>

[Back to TOC](#top)

In the next few cells, we will compare the methods you've implemented. Helper functions are provided for you, which you can use to make some conclusions. You can modify the code as needed!

First, let's have a function that plots the average scores of relevant (levels 3 and 4) and non-relevant (levels 0, 1, and 2) scores in terms of training epochs for different loss functions:

In [ ]:
loss_functions = {'pointwise':[pointwise_loss, train_batch],
                 'pairwise':[compute_lambda_i, train_batch_vector],
                 'listwise':[listwise_loss, train_batch_vector]}

def plot_relevance_scores(batches, loss_function):
    seed(420)
    net = NeuralModule()
    optimizer = Adam(net.parameters(), lr=0.005)
    loss_fn = loss_functions[loss_function][0]
    train_fn = loss_functions[loss_function][1]
    
    
    train_batchs = batches[:len(batches)*3//4]
    test_batchs = batches[len(batches)*3//4:]
    
    rel, nrel = [], []
    
    for i in range(100):
        r, n = [], []
        for x, y in test_batchs:
            binary_rel = np.round(y/4,0)
            scores = net(x)[:,0]
            r.append(torch.sum(scores * binary_rel).detach().numpy() / torch.sum(binary_rel).detach().numpy())
            n.append(torch.sum(scores * (1. - binary_rel)).detach().numpy() / torch.sum((1. - binary_rel)).detach().numpy())
            
        for x, y in train_batchs:
            train_fn(net, x, y, loss_fn, optimizer)
        rel.append(np.mean(np.array(r)))
        nrel.append(np.mean(np.array(n)))
        
    
        
    plt.figure()
    plt.suptitle(loss_function)
    plt.plot(np.arange(10,len(rel)), rel[10:], label='relevant')
    plt.plot(np.arange(10,len(nrel)), nrel[10:], label='non-relevant')
    plt.legend()
    
        

For efficiency issues, we select a small number (83) of queries to test different loss functions.
We split these queries into train and test with a 3:1 ratio.

In [ ]:
batches = [train_data[i] for i in [181, 209, 233, 242, 259, 273, 327, 333, 377, 393, 410, 434, 452, 503, 529, 573, 581, 597, 625, 658, 683, 724, 756, 757, 801, 825, 826, 828, 874, 902, 1581, 1588, 1636, 1691, 1712, 1755, 1813, 1983, 2001, 2018, 2021, 2024, 2029, 2065, 2095, 2100, 2171, 2172, 2174, 2252, 2274, 2286, 2288, 2293, 2297, 2353, 2362, 2364, 2365, 2368, 2400, 2403, 2433, 2434, 2453, 2472, 2529, 2534, 2539, 2543, 2555, 2576, 2600, 2608, 2636, 2641, 2653, 2692, 2714, 2717, 2718, 2723, 2724]]

Next, we train a neural network with different loss functions on the selected queries.
During training, we save the average scores of relevant and non-relevant validation items for each training epoch and plot them as follows:

In [ ]:

# plot_relevance_scores(batches, 'pointwise')

# plot_relevance_scores(batches, 'pairwise')

plot_relevance_scores(batches, 'listwise')

scores:  tensor([[-0.0519],
        [ 0.0514],
        [-0.0460],
        [-0.0454],
        [-0.0194],
        [-0.0024],
        [-0.1094],
        [ 0.0139],
        [-0.0255],
        [-0.1121],
        [-0.0432],
        [-0.0071],
        [-0.0566],
        [-0.0361],
        [-0.0641],
        [-0.0482],
        [-0.0655],
        [-0.0681],
        [-0.0572],
        [-0.0984],
        [-0.0819],
        [-0.0928],
        [ 0.0025],
        [-0.0627],
        [-0.0870],
        [-0.0770],
        [ 0.0054],
        [-0.0946],
        [-0.0947],
        [-0.0188],
        [-0.0834],
        [ 0.0181],
        [-0.0024],
        [-0.1057],
        [-0.0041],
        [ 0.0044],
        [ 0.0374],
        [-0.0461],
        [-0.0111],
        [-0.0397]], grad_fn=<AddmmBackward>)
labels:  tensor([2., 2., 0., 1., 1., 3., 1., 4., 1., 1., 2., 3., 2., 2., 3., 1., 2., 3.,
        2., 2., 3., 2., 2., 1., 3., 0., 2., 0., 2., 0., 4., 4., 2., 1., 0., 2.,
        4., 0., 3., 3.])
ideal_label

TypeError: ignored

In [ ]:
torch.cuda.is_available()

**Implementation (15 points):**
Now implement a function similar to `plot_relevance_scores` that measures the NDCG@10 on the test split with different loss functions.
Train your model for 10 epochs.
For NDCG@10 use `evaluate.ndcg10(scores.detach().numpy(), y.detach().numpy())` for each query and average through all queries to obtain NDCG@10 for each loss function at each epoch.

In [ ]:
# TODO: Implement this! (15 points)


def plot_ndcg10(batches, loss_function):
    seed(420)
    net = NeuralModule()
    optimizer = Adam(net.parameters(), lr=0.005)
    loss_fn = loss_functions[loss_function][0]
    train_fn = loss_functions[loss_function][1]
        
    train_batchs = batches[:len(batches)*3//4]
    test_batchs = batches[len(batches)*3//4:]
    
    ndcg = []
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
#     plt.figure()
    plt.plot(np.arange(len(ndcg)), ndcg, label=loss_function)
    plt.legend()
    
        
    


In [ ]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [ ]:

plot_ndcg10(batches, 'pointwise')

plot_ndcg10(batches, 'pairwise')

plot_ndcg10(batches, 'listwise')

Write a conclusion in the next cell, considering (40 points):
- rates of convergence
- time complexity
- distinguishing relevant and non-relevant items
- performance for low data wrt NDCG@10
- performance across queries
- ... any other observations

YOUR ANSWER HERE